In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Embedding 
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [2]:
df=pd.read_csv('../input/fake-news/train.csv')
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
df=df.dropna()
df.shape

(18285, 5)

In [6]:
X=df.drop('label', axis=1)
y=df['label']

In [7]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
msg=X.copy()

In [10]:
msg.head(8)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...


In [11]:
msg.reset_index(inplace=True)
msg.head(8)

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
6,7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
7,9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...


In [12]:
ps=PorterStemmer()
corpus=[]

uniq_wrd=set()
temp_set=set()
temp_list=[]

for i in range(len(msg)):
    review= re.sub('[^a-zA-Z0-9]',' ',msg['title'][i])
    review=review.lower()
    review=review.split()
    
    temp_set=set(review)
    uniq_wrd=uniq_wrd.union(temp_set)   
    temp_list.append(review)
    
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [13]:
len(corpus)
corpus[:6]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 '15 civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart']

In [14]:


len(uniq_wrd)
    

20314

In [15]:
onehot_repr=[one_hot(words,10000) for words in corpus]
onehot_repr

[[8438, 3706, 6894, 8277, 3944, 1219, 3542, 7753, 3264, 5400],
 [9580, 4353, 5296, 3442, 1155, 7090, 971],
 [8195, 6309, 2635, 2189],
 [690, 4874, 6903, 3835, 2560, 7219, 8992],
 [5145, 1155, 4388, 1004, 8843, 6677, 1155, 9265, 193, 2804],
 [7398,
  3511,
  4479,
  1952,
  8035,
  8159,
  2572,
  914,
  7900,
  4711,
  2677,
  906,
  6272,
  2085,
  971],
 [5000, 1296, 9545, 3023, 1579, 595, 7167, 677, 6654, 2889, 3090],
 [9629, 7789, 9949, 1568, 5091, 860, 8159, 4971, 6654, 2889, 3090],
 [2139, 6498, 5470, 5779, 9680, 5074, 9687, 1154, 8159, 2254],
 [6770, 922, 1326, 1328, 9582, 2138, 1512, 6091],
 [8765, 964, 5442, 5382, 6174, 3285, 8473, 2316, 8382, 3830, 8022],
 [2560, 3043, 3944, 5074, 8159, 5091],
 [2495, 5057, 6355, 315, 7730, 4805, 7890, 8138, 3487],
 [6038, 1179, 1192, 9717, 9073, 6957, 3091, 6654, 2889, 3090],
 [4830, 2105, 773, 5807, 2095, 6654, 2889, 3090],
 [2714, 6993, 3168, 2104, 654, 1624, 2608, 1115, 8260, 3832, 6305, 7313],
 [9119, 2001, 4353],
 [2058, 5625, 8261, 358

In [16]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

embedded_docs

array([[   0,    0,    0, ..., 7753, 3264, 5400],
       [   0,    0,    0, ..., 1155, 7090,  971],
       [   0,    0,    0, ..., 6309, 2635, 2189],
       ...,
       [   0,    0,    0, ..., 6654, 2889, 3090],
       [   0,    0,    0, ..., 6228, 8358, 5953],
       [   0,    0,    0, ..., 2893, 4349, 5801]], dtype=int32)

In [17]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 8438,
       3706, 6894, 8277, 3944, 1219, 3542, 7753, 3264, 5400], dtype=int32)

In [18]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(10000,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

2022-10-17 19:02:21.692579: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-17 19:02:21.790603: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-17 19:02:21.791425: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-17 19:02:21.793456: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            400000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 513,001
Trainable params: 513,001
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [20]:
## train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)


In [21]:
## Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10


2022-10-17 19:02:24.975196: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-10-17 19:02:28.670606: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


383/383 [==============================] - 8s 9ms/step - loss: 0.2627 - accuracy: 0.8811 - val_loss: 0.1890 - val_accuracy: 0.9185
Epoch 2/10
383/383 [==============================] - 3s 7ms/step - loss: 0.1168 - accuracy: 0.9555 - val_loss: 0.2061 - val_accuracy: 0.9171
Epoch 3/10
383/383 [==============================] - 3s 8ms/step - loss: 0.0668 - accuracy: 0.9775 - val_loss: 0.2324 - val_accuracy: 0.9178
Epoch 4/10
383/383 [==============================] - 3s 7ms/step - loss: 0.0305 - accuracy: 0.9913 - val_loss: 0.2894 - val_accuracy: 0.9171
Epoch 5/10
383/383 [==============================] - 3s 8ms/step - loss: 0.0176 - accuracy: 0.9949 - val_loss: 0.4054 - val_accuracy: 0.9160
Epoch 6/10
383/383 [==============================] - 3s 7ms/step - loss: 0.0181 - accuracy: 0.9947 - val_loss: 0.3629 - val_accuracy: 0.9092
Epoch 7/10
383/383 [==============================] - 3s 8ms/step - loss: 0.0124 - accuracy: 0.9962 - val_loss: 0.4487 - val_accuracy: 0.9168
Epoch 8/10
383/38

In [22]:
y_pred=model.predict(X_test)
y_pred

array([[9.9999022e-01],
       [1.3518512e-06],
       [9.9269431e-03],
       ...,
       [1.8038019e-05],
       [9.9990940e-01],
       [9.9999368e-01]], dtype=float32)

In [23]:
import numpy as np
from sklearn.metrics import confusion_matrix,accuracy_score

In [24]:
y_pred_06=np.where(y_pred>=0.5,1,0)
print(confusion_matrix(y_test,y_pred_06))
print(accuracy_score(y_test,y_pred_06))

[[3095  324]
 [ 208 2408]]
0.911847555923778
